In [1]:
import re
import pandas as pd
from sqlglot import parse_one, exp, transpile
from datatrace.plsqlstatement import PlsqlStatement
from datatrace.datatrace import DataTrace
import datatrace.params as params

pd.set_option('display.max_columns', None)

In [2]:
# script_path='tests\\resources\\sql_cases\\Proceso_Perdidas_Fraude.sql'
# script_path='tests\\resources\\sql_cases\\3.EP_FFVV_TRADICIONAL 2.sql'
script_path='tests\\resources\\sql_cases\\0. Package BDI 2 copy.sql'
output_path="tests/resources/output/DataTrace_reportv2.xlsx"
employee_code="t66001"

In [3]:
general_list = []
dt = DataTrace()
# df_statements, df_comments, plsql_script_dict = dt.extract_statements(script_path)
# general_list.append(plsql_script_dict)

file_plsql_script = open(script_path, 'r', errors='ignore')
plsql_script = "".join(file_plsql_script.readlines())

comments, clean_plsql_script = dt.extract_and_clean_comments(plsql_script)


In [4]:
#statement_pattern = re.compile(r"(" + "|".join(list(params.statement_type_dict.keys())) + r")", re.DOTALL | re.IGNORECASE)
#all_statements = statement_pattern.findall(clean_plsql_script)
all_statements = dt.get_statements_by_pattern_dict(clean_plsql_script)

2025-06-03 19:08:08,794 - datatrace - INFO: Empiezo debug pattern DECLARE_PROCEDURE - T66001
2025-06-03 19:08:08,795 - datatrace - INFO: Empiezo debug child SELECT_INTO - T66001
2025-06-03 19:08:08,796 - datatrace - INFO: Empiezo debug child MERGE_INTO - T66001
2025-06-03 19:08:08,796 - datatrace - INFO: Empiezo debug child EXECUTE_IMMEDIATE - T66001


<re.Match object; span=(244, 313), match="SELECT SYS_CONTEXT('USERENV', 'SESSION_USER') INT>
<re.Match object; span=(327, 1839), match="MERGE INTO USRDB_CRM_SOLTEC.CT_TIPVENTATABLA CT' >
<re.Match object; span=(1844, 1867), match='EXECUTE IMMEDIATE CSQL;'>


In [ ]:
import re
def extraer_procedures(plsql_script):
    procedures = []
    lines = plsql_script.splitlines()
    in_proc = False
    proc_lines = []
    proc_name = None
    begin_count = 0
    end_count = 0
    for line in lines:
        stripped = line.strip()
        # Detecta inicio de un PROCEDURE
        if not in_proc:
            match = re.match(r"(?i)\bPROCEDURE\s+(\w+)", stripped)
            if match:
                proc_name = match.group(1)
                in_proc = True
                proc_lines = [line]
                begin_count = 0
                end_count = 0
                continue
        if in_proc:
            proc_lines.append(line)
            if re.search(r"(?i)\bBEGIN\b", stripped):
                begin_count += 1
            if re.search(r"(?i)\bEND\b", stripped):
                end_count += 1
            # Detecta fin de PROCEDURE si encuentra END <nombre>;
            if (
                begin_count > 0 and
                re.search(rf"(?i)\bEND\s+{proc_name}\s*;", stripped)
            ):
                procedures.append('\n'.join(proc_lines))
                in_proc = False
                proc_name = None
                proc_lines = []
    return procedures

In [11]:
script = """
DECLARE

TABLA_SEGURO_SPT VARCHAR2(50):='T44601.DE_SEGUROS_MIS_BLOQUE01';
TABLA_SEGURO_RET VARCHAR2(50):='T44601.MM_SEGRETORNO_MIS2';
TABLA_SEGURO_MUL VARCHAR2(50):='T44601.DE_SEGUROS_MIS_BLOQUE02';
CLU_MES NUMBER;

PROCEDURE P_DE_BENEFICIOS
IS
BEGIN

	BEGIN
	
		BEGIN
			EXECUTE IMMEDIATE
			'DROP TABLE TP_EP_BENEFICIOS_INV';
			EXCEPTION
			WHEN OTHERS THEN
				IF SQLCODE != -942 THEN 
					RAISE;
				END IF;    
		END;
		
		EXECUTE IMMEDIATE '
		CREATE TABLE TP_EP_BENEFICIOS_INV TABLESPACE D_CRM AS
		SELECT A.CODLOTEOFERTA, A.CODCLAVECIC, A.MTOEP, A.PCTTEA, 
		A.FLGLEADAPROBADO, A.FLGTIENEAMPLIACION, A.DESTARJETA, A.DESPLASTICO, A.NUMBEHAVIORSCORE,
		CASE WHEN A.PCTTEA<=0.2 THEN ''RT2''
			 WHEN A.PCTTEA>0.2 AND A.PCTTEA<=0.45 THEN ''RT1''
			 WHEN A.PCTTEA>0.45 THEN ''RT0'' END RANGO_TASA,
		0 FLG_MILLAS,0 CANT_MILLAS,''.'' BONO_FICHA
		FROM UM_EP_PORTAFOLIO_INV A
		LEFT JOIN (SELECT DISTINCT Z.CODCLAVECIC,Z.CODMES,Z.NUEVA_SEGMENTACION,Z.CODSUBSEGMENTO FROM PROY_AUTO_D.HM_NUEVASEG_CONSUMO_2 Z
		WHERE Z.CODMES='||CLU_MES||' AND Z.NUEVA_SEGMENTACION = ''ARMANDO'' AND Z.CODSUBSEGMENTO = ''M1N'') C ON A.CODCLAVECIC = C.CODCLAVECIC
		';
	END;

	BEGIN
		BEGIN
			EXECUTE IMMEDIATE
			'DROP TABLE DE_EP_BENEFICIOS_INV';
			EXCEPTION
			WHEN OTHERS THEN
				IF SQLCODE != -942 THEN 
					RAISE;
				END IF;    
		END;
		
		EXECUTE IMMEDIATE '
		CREATE TABLE DE_EP_BENEFICIOS_INV TABLESPACE D_CRM AS
		SELECT A.*, 
			   CASE WHEN A.FLG_MILLAS=1 THEN ''EPMLL'' ELSE ''.'' END TIPBENEFICIO,
		CASE WHEN A.FLG_MILLAS=1 THEN A.BONO_FICHA
			 ELSE NULL END VALORCAMPOVAR19
		FROM TP_EP_BENEFICIOS_INV A
		';
	END;
END P_DE_BENEFICIOS;

FUNCTION CREARTABLALOG(PNOMBRETABLA VARCHAR2) RETURN VARCHAR2 IS
    EXISTETABLALOG VARCHAR2(50);
    NOMBRETABLALOG VARCHAR2(50);
  BEGIN
    NOMBRETABLALOG := SUBSTR('LOG_' || PNOMBRETABLA, 1, 30);
    SELECT MAX(TABLE_NAME)
      INTO EXISTETABLALOG
      FROM USER_TABLES
     WHERE TABLE_NAME = UPPER(NOMBRETABLALOG);
  
    /* Crear tabla*/
    IF (EXISTETABLALOG IS NOT NULL) THEN
      EXECUTE IMMEDIATE 'TRUNCATE TABLE ' || NOMBRETABLALOG || ' ';
      EXECUTE IMMEDIATE 'DROP TABLE ' || NOMBRETABLALOG || ' ';
    END IF;
    EXECUTE IMMEDIATE 'CREATE TABLE ' || NOMBRETABLALOG || ' (
                      CODLOTEOFERTA                      NUMBER,
                      TIPVENTA                           NUMBER,
                      TIPOFERTA                          NUMBER,
					            CODIGOLOG                          CHAR(4),
                      DESMENSAJE                         VARCHAR2(500),
                      TIPMENSAJE                         CHAR(1),
                      FECINICIO                          DATE
                    ) TABLESPACE D_CRM';
    RETURN NOMBRETABLALOG;
  END CREARTABLALOG;

PROCEDURE P_SEGUROS
IS
BEGIN
	EXECUTE IMMEDIATE '
	MERGE INTO UM_EP_PORTAFOLIO_INV EP
	USING (
		SELECT A.CODCLAVECIC,
		F_SEGUROS_PRIORIDAD(SPT.GRUPOPRIORIDAD, RET.GRUPOPRIORIDAD, MUL.GRUPOPRIORIDAD) AS LEADSEGURO
		FROM (SELECT * FROM UM_EP_PORTAFOLIO_INV) A
		LEFT JOIN '||TABLA_SEGURO_SPT||' SPT 
		ON A.CODCLAVECIC = SPT.CODCLAVECIC
		LEFT JOIN '||TABLA_SEGURO_MUL||' MUL 
		ON A.CODCLAVECIC = MUL.CODCLAVECIC
		LEFT JOIN '||TABLA_SEGURO_RET||' RET 
		ON A.CODCLAVECIC = RET.CODCLAVECIC
	) SEGURO 
	ON (EP.CODCLAVECIC=SEGURO.CODCLAVECIC)
	WHEN MATCHED THEN UPDATE SET
	EP.LEADSEGURO = SEGURO.LEADSEGURO
	';
	COMMIT;
	
END P_SEGUROS;

BEGIN

SELECT MAX(CODMES) INTO CLU_MES FROM PROY_AUTO_D.HM_NUEVASEG_CONSUMO_2;
P_DE_BENEFICIOS;
P_SEGUROS;

END;
"""
procs = extraer_procedures(script)
for i, proc in enumerate(procs, 1):
    print(f"\n>> PROCEDURE {i}:\n{proc}")


>> PROCEDURE 1:
PROCEDURE P_DE_BENEFICIOS
IS
BEGIN

	BEGIN

		BEGIN
			EXECUTE IMMEDIATE
			'DROP TABLE TP_EP_BENEFICIOS_INV';
			EXCEPTION
			WHEN OTHERS THEN
				IF SQLCODE != -942 THEN 
					RAISE;
				END IF;    
		END;

		EXECUTE IMMEDIATE '
		CREATE TABLE TP_EP_BENEFICIOS_INV TABLESPACE D_CRM AS
		SELECT A.CODLOTEOFERTA, A.CODCLAVECIC, A.MTOEP, A.PCTTEA, 
		A.FLGLEADAPROBADO, A.FLGTIENEAMPLIACION, A.DESTARJETA, A.DESPLASTICO, A.NUMBEHAVIORSCORE,
		CASE WHEN A.PCTTEA<=0.2 THEN ''RT2''
			 WHEN A.PCTTEA>0.2 AND A.PCTTEA<=0.45 THEN ''RT1''
			 WHEN A.PCTTEA>0.45 THEN ''RT0'' END RANGO_TASA,
		0 FLG_MILLAS,0 CANT_MILLAS,''.'' BONO_FICHA
		FROM UM_EP_PORTAFOLIO_INV A
		LEFT JOIN (SELECT DISTINCT Z.CODCLAVECIC,Z.CODMES,Z.NUEVA_SEGMENTACION,Z.CODSUBSEGMENTO FROM PROY_AUTO_D.HM_NUEVASEG_CONSUMO_2 Z
		WHERE Z.CODMES='||CLU_MES||' AND Z.NUEVA_SEGMENTACION = ''ARMANDO'' AND Z.CODSUBSEGMENTO = ''M1N'') C ON A.CODCLAVECIC = C.CODCLAVECIC
		';
	END;

	BEGIN
		BEGIN
			EXECUTE IMMEDIATE
			'DR

In [3]:
import sqlglot

# Sentencia SQL de tipo UPDATE más compleja
sql = """
UPDATE mm_leads_Crma_agentes A SET A.PRIORIZACION_LEAD = CASE 
WHEN TIPVENTA = 43 THEN 1
WHEN TIPVENTA IN (6,110) THEN 2 
WHEN TIPVENTA IN (17,18) THEN 3
WHEN TIPVENTA IN (54) THEN 4
END ;
"""

# Parsear la sentencia SQL
parsed = sqlglot.parse_one(sql, read="oracle")

# Mostrar el árbol de sintaxis abstracta (AST)
print(parsed)

UPDATE mm_leads_Crma_agentes AS A SET A.PRIORIZACION_LEAD = CASE WHEN TIPVENTA = 43 THEN 1 WHEN TIPVENTA IN (6, 110) THEN 2 WHEN TIPVENTA IN (17, 18) THEN 3 WHEN TIPVENTA IN (54) THEN 4 END


In [ ]:
import sqlglot

# Sentencia SQL de tipo UPDATE
sql = "UPDATE mm_leads_Crma_agentes AS A SET A.PRIORIZACION_LEAD = CASE WHEN TIPVENTA = 43 THEN 1 WHEN TIPVENTA IN (6, 110) THEN 2 WHEN TIPVENTA IN (17, 18) THEN 3 WHEN TIPVENTA IN (54) THEN 4 END"

# Parsear la sentencia SQL
parsed = sqlglot.parse_one(sql)

# Generar la sentencia SQL desde el AST
generated_sql = parsed.sql()

print(generated_sql)

UPDATE mm_leads_Crma_agentes AS A SET A.PRIORIZACION_LEAD = CASE WHEN TIPVENTA = 43 THEN 1 WHEN TIPVENTA IN (6, 110) THEN 2 WHEN TIPVENTA IN (17, 18) THEN 3 WHEN TIPVENTA IN (54) THEN 4 END


In [17]:
import sqlglot
from sqlglot import exp

# Sentencia SQL de tipo UPDATE
sql = """
UPDATE employees
SET salary = salary * 1.1,
    bonus = bonus + 1000
FROM employees e
JOIN departments d ON e.department_id = d.id
WHERE d.name = 'Sales';
"""

sql = """UPDATE FORMATO_BDI_202410 x SET x.flggrupocontrolcliente='N' where flggrupocontrolcliente is null;"""

# Parsear la sentencia SQL
parsed = sqlglot.parse_one(sql, read="oracle")

# Extraer información del AST
update_statement = parsed.find(exp.Update)
output_table = update_statement.this.name
set_clauses = update_statement.expressions
from_clause = update_statement.args.get('from')
where_clause = update_statement.args.get('where')

# Mostrar la información extraída
print(f"Output Table: {output_table}")
print("Set Clauses:")
for clause in set_clauses:
    output_field = clause.this.name
    input_expression = clause.expression
    input_field = input_expression.this.name if isinstance(input_expression, exp.Column) else None
    print(f"  Output Field: {output_field}, Input Field: {input_field}, Expression: {input_expression}")

if from_clause:
    print(f"From Clause: {from_clause}")

if where_clause:
    print(f"Where Clause: {where_clause}")

if update_statement:
    print(f"Update Clause: {update_statement}")

Output Table: FORMATO_BDI_202410
Set Clauses:
  Output Field: flggrupocontrolcliente, Input Field: None, Expression: 'N'
Where Clause: WHERE flggrupocontrolcliente IS NULL
Update Clause: UPDATE FORMATO_BDI_202410 AS x SET x.flggrupocontrolcliente = 'N' WHERE flggrupocontrolcliente IS NULL


In [25]:
import sqlglot
from sqlglot import exp

# Sentencia SQL de tipo UPDATE
sql = """
UPDATE USRDB_CRM_SOLTEC.CT_CONTROLCARGASANDBOXALAKE
	SET FLGESTADO='S',
  FECEJECUCION = SYSDATE
	WHERE
	NUMIDENTIFICADOR=47;
"""

sql2 = """
UPDATE (SELECT * FROM MD_LEADSAGENDA B WHERE B.PRODUCTO IN('CREDITO EFECTIVO','BALANCE TRANSFER','CUENTA SUELDO','PRESTAMO TARJETERO','TARJETA DE CREDITO')) A
  SET A.PRODUCTO = 'Este cliente es tu CASERO y tiene un A.PRODUCTO APROVÉCHALO!'
  WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD','CASERO VENTA','CUADERNITO VIRTUAL'));
"""

# Parsear la sentencia SQL
parsed = sqlglot.parse_one(sql)

# Extraer información del AST
update_statement = parsed.find(exp.Update)
table = update_statement.this.name
set_clauses = update_statement.expressions
from_clause = update_statement.args.get('from')
where_clause = update_statement.args.get('where')

# Mostrar la información extraída
print(f"Table: {table}")
print("Set Clauses:")
for clause in set_clauses:
    print(f"  {clause.this} = {clause.expression}")

if from_clause:
    print(f"From Clause: {from_clause}")

if where_clause:
    print(f"Where Clause: {where_clause}")

Table: CT_CONTROLCARGASANDBOXALAKE
Set Clauses:
  FLGESTADO = 'S'
  FECEJECUCION = SYSDATE
Where Clause: WHERE NUMIDENTIFICADOR = 47


In [165]:
import sqlglot
from sqlglot import exp
from datatrace.base.statements.select_statement import BasicSelectStatement

# Sentencia SQL de tipo UPDATE
sql = """
UPDATE (SELECT * FROM MD_LEADSAGENDA B WHERE B.PRODUCTO IN('CREDITO EFECTIVO','BALANCE TRANSFER','CUENTA SUELDO','PRESTAMO TARJETERO','TARJETA DE CREDITO')) A
  SET A.PRODUCTO = 'Este cliente es tu CASERO y tiene un '||A.PRODUCTO||' APROVÉCHALO!'
  WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD','CASERO VENTA','CUADERNITO VIRTUAL'));
"""

sql2 = """
UPDATE USRDB_CRM_SOLTEC.CT_CONTROLCARGASANDBOXALAKE B
	SET FLGESTADO='S',
  FECEJECUCION = SYSDATE
	WHERE
	NUMIDENTIFICADOR=47;
"""

# Parsear la sentencia SQL
parsed = sqlglot.parse_one(sql, dialect="oracle")

# Extraer información del AST
update_statement = parsed.find(exp.Update)
output_table = update_statement.this.name if update_statement.this else None
set_clauses = update_statement.expressions
subquery = update_statement.args.get('this')
where_clause = update_statement.args.get('where')

# alias = update_statement.args.get('this').db
print("alias :",update_statement.this.alias)

# Mostrar la información extraída
print(f"Output Table: {output_table}")
print("Set Clauses:")
for clause in set_clauses:
    output_field = clause.this.name
    input_expression = clause.expression
    input_field = input_expression.this.name if isinstance(input_expression, exp.Column) else None
    print(f"Output Field: {output_field}, Input Field: {input_field}, Expression: {input_expression}")

if subquery:
    print(f"Subquery: {subquery}")    

if where_clause:
    print(f"Where Clause: {where_clause}")

alias : A
Output Table: 
Set Clauses:
Output Field: PRODUCTO, Input Field: None, Expression: 'Este cliente es tu CASERO y tiene un ' || A.PRODUCTO || ' APROVÉCHALO!'
Subquery: (SELECT * FROM MD_LEADSAGENDA AS B WHERE B.PRODUCTO IN ('CREDITO EFECTIVO', 'BALANCE TRANSFER', 'CUENTA SUELDO', 'PRESTAMO TARJETERO', 'TARJETA DE CREDITO')) AS A
Where Clause: WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD', 'CASERO VENTA', 'CUADERNITO VIRTUAL'))


In [50]:
STATEMENT_STRUCT_DICT = {
                        "CREATE_TABLE_AS": {
                            "pattern": r"CREATE\s+TABLE\s+(?P<raw_output_table>.+?)\s+(:?TABLESPACE)?\s*(?P<tablespace>.+?)?\s*\bAS\b\s*(?P<raw_select_statement>[^;]+);\s*",
                            "struct_list": ["raw_output_table", "tablespace", "raw_select_statement"]
                        },
                        "INSERT_INTO_SELECT": {
                            #"pattern": r"INSERT\s+INTO\s+(?P<raw_output_table>[^\s(]+)(?:\s+NOLOGGING)?\s*(?P<raw_output_table_struct>\([^\)]*\))?\s*(?P<raw_select_statement>SELECT[\s\S]+?);",
                            "pattern": r"INSERT\s+INTO\s+(?P<raw_output_table>[^\s(]+)(?:\s+NOLOGGING)?\s*(?P<raw_output_table_struct>\([^\)]*\))?\s*(?P<raw_select_statement>\(?\s*SELECT[\s\S]+?\)?);",
                            "struct_list": ["raw_output_table", "raw_output_table_struct", "raw_select_statement"]
                        },
                        "SPOOL_SELECT":{
                            #"pattern": r"SPOOL\s+(?P<raw_output_table>[^\s(]+)\s*(?P<raw_select_statement>SELECT[\s\S]+?);",
                            "pattern": r"SPOOL\s+(?P<raw_output_table>[^\s;]+)\s*\n(?P<raw_select_statement>(?:\s*SELECT[\s\S]+?)+?)^\s*SPOOL\s+(?:OFF|OUT)\s*;",
                            "struct_list": ["raw_output_table", "raw_select_statement"]
                        },

                        "UPDATE":{
                            #"pattern": r"SPOOL\s+(?P<raw_output_table>[^\s(]+)\s*(?P<raw_select_statement>SELECT[\s\S]+?);",
                            "pattern": r"UPDATE\s+(?P<raw_output_table>.+?)\s+SET\s+(?P<set_clause>.+?)\s+WHERE\s+(?P<where_clause>.+?);",
                            "struct_list": ["raw_output_table", "set_clause", "where_clause"]
                        }
                      }

In [1]:
#from datatrace.base.statements.generator_statement import GeneratorStatement
from sqlglot import parse_one, exp

class UpdateStatement():
    def __init__(self, raw_statement, statement_id=0):
        """
        Initialize the UpdateStatement with the raw SQL statement and an optional statement ID.
        """
        self.raw_statement = raw_statement
        self.field_mapping_exception = None
        self.statement_id = statement_id
        self.statement = self.parse_statement()
        self.statement_components_dict = self.get_components()
        self.output_table = self.get_output_table()

    def parse_statement(self):
        """
        """
        try:
            if self.raw_statement:
                return parse_one(self.raw_statement, dialect="oracle")
            else:
                self.field_mapping_exception = "Source raw query is null."
                return None
        except Exception as e:
            self.field_mapping_exception = str(e)
            return None
        
    def get_components(self) -> dict():
        """
        """
        statement_components_dict = dict()
        if self.statement:
            parsed_dict = self.statement.find(exp.Update)
            statement_components_dict = {
                "output_table": parsed_dict.args.get("this").name if parsed_dict.args.get("this") else None,
                "subquery_table": parsed_dict.args.get('this'),
                "output_table_alias": parsed_dict.this.alias if parsed_dict.this.alias else None,
                "set_clause": parsed_dict.expressions,
                "where_clause": parsed_dict.args.get("where")
            }

        return statement_components_dict

    def get_output_table(self) -> str:
        """
        """
        return self.statement_components_dict.get("output_table") if self.statement_components_dict.get("output_table") else self.statement_components_dict.get("subquery_table")

    def get_output_table_alias(self) -> str:
        return self.statement_components_dict.get("output_table_alias")

    def get_field_mapping(self) -> dict:
        """
        Get the field mapping from the UPDATE statement.
        """
        field_mapping_dict = {
            "field_mapping_list": [],
            "flag_mapping_success": 0,
            "field_mapping_exception": None
        }
        try:
            if self.statement_components_dict.get("set_clause"):
                set_clause = self.statement_components_dict.get("set_clause")
                field_mapping_list = self.parse_set_clause(set_clause)
                field_mapping_dict["field_mapping_list"] = field_mapping_list
                field_mapping_dict["flag_mapping_success"] = 1

            return field_mapping_dict
        except Exception as e:
            return {
                "field_mapping_list": [],
                "flag_mapping_success": 0,
                "field_mapping_exception": str(e)
            }

    def parse_set_clause(self, set_clause: str) -> list:
        """
        Parse the SET clause to extract field mappings.
        """
        field_mapping_list = []
        for clause in self.statement_components_dict.get("set_clause"):
            field_mapping_list.append({
                "output_field": clause.this.name,
                "input_field" : clause.expression.this.name if isinstance(clause.expression, exp.Column) else None,
                "input_expression": f"{clause.expression}",
                "filter_expression": f"{self.statement_components_dict.get("where_clause").this}"
            })
        return field_mapping_list

In [3]:
####NUEVA VERSION
from src.main.datatrace.base.statements.generator_statement import GeneratorStatement
from sqlglot import parse_one, exp

class UpdateStatement(GeneratorStatement):
    def __init__(self, raw_statement, statement_id=0):
        """
        Initialize the UpdateStatement with the raw SQL statement and an optional statement ID.
        """
        self.raw_statement = raw_statement
        self.field_mapping_exception = None
        self.statement_id = statement_id
        self.statement = self.parse_statement()
        self.statement_components_dict = self.get_components()
        self.output_table = self.get_output_table()

    def parse_statement(self):
        """
        """
        try:
            if self.raw_statement:
                return parse_one(self.raw_statement, dialect="oracle")
            else:
                self.field_mapping_exception = "Source raw query is null."
                return None
        except Exception as e:
            self.field_mapping_exception = str(e)
            return None
        
    def get_components(self) -> dict():
        """
        """
        statement_components_dict = dict()
        if self.statement:
            parsed_dict = self.statement.find(exp.Update)
            statement_components_dict = {
                "output_table": parsed_dict.args.get("this"),
                "output_field_alias": parsed_dict.this.alias if parsed_dict.this.alias else None,
                "set_clause": parsed_dict.expressions,
                "where_clause": parsed_dict.args.get("where")
            }

        return statement_components_dict

    def get_output_table(self) -> str:
        """
        """
        output_table_component = self.statement_components_dict.get("output_table")
        if output_table_component:
            if isinstance(output_table_component, exp.Table):
                return output_table_component.name
            elif isinstance(output_table_component, exp.Subquery):
                return output_table_component.sql(dialect="oracle")

        return output_table_component

    def get_output_table_alias(self) -> str:
        return self.statement_components_dict.get("output_table_alias")

    def get_field_mapping(self) -> dict:
        """
        Get the field mapping from the UPDATE statement.
        """
        field_mapping_dict = {
            "field_mapping_list": [],
            "flag_mapping_success": 0,
            "field_mapping_exception": None
        }
        try:
            if self.statement_components_dict.get("set_clause"):
                set_clause = self.statement_components_dict.get("set_clause")
                field_mapping_list = self.parse_set_clause(set_clause)
                field_mapping_dict["field_mapping_list"] = field_mapping_list
                field_mapping_dict["flag_mapping_success"] = 1

            return field_mapping_dict
        except Exception as e:
            return {
                "field_mapping_list": [],
                "flag_mapping_success": 0,
                "field_mapping_exception": str(e)
            }

    def parse_set_clause(self, set_clause: str) -> list:
        """
        Parse the SET clause to extract field mappings.
        """
        field_mapping_list = []
        for clause in self.statement_components_dict.get("set_clause"):
            field_mapping_list.append({
                "clause": "update_set",
                "output_table": self.output_table,
                "output_field_alias" : clause.this.name,
                "expression_sql": f"{clause.sql(dialect='oracle')}",
                "input_field": None#f"{self.statement_components_dict.get("where_clause").this}"
            })
        return field_mapping_list

In [ ]:
# Example usage
raw_update_statement = """
UPDATE (SELECT * FROM MD_LEADSAGENDA B WHERE B.PRODUCTO IN('CREDITO EFECTIVO','BALANCE TRANSFER','CUENTA SUELDO','PRESTAMO TARJETERO','TARJETA DE CREDITO')) A
  SET A.PRODUCTO = 'Este cliente es tu CASERO y tiene un A.PRODUCTO APROVÉCHALO!'
  WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD','CASERO VENTA','CUADERNITO VIRTUAL'));
"""

raw_update_statement2 = """
UPDATE USRDB_CRM_SOLTEC.CT_CONTROLCARGASANDBOXALAKE
	SET FLGESTADO='S',
  FECEJECUCION = SYSDATE
	WHERE
	NUMIDENTIFICADOR=47;
"""

raw_update_statement3 = """
UPDATE AR03078_TP_DETA_FINAL_LETRAS A
  SET A.ESTADO_BAJA = 'C',
  A.FECHA_BAJA = TRUNC(SYSDATE)
  WHERE EXISTS (SELECT 1
  FROM AR03078_TP_DETA_LTR_CANCELADA B
  WHERE A.CODCLAVEOPECTA = B.CODCLAVEOPECTA
  AND A.NUMERO_UNICO = B.NUMERO_UNICO)
  AND A.ESTADO_BAJA IS NULL;
"""

raw_update_statement4 = """
UPDATE hd_leadsagendacumulado A
SET FLG_GATILLO = 'S'
WHERE UPPER(TIPO_LEAD) LIKE '%GATILLO%' OR UPPER(TIPO_LEAD) LIKE '%QUIERO%';"""

raw_update_statement5 = """
UPDATE PARALELA_FT_CEFCNV_TABLON SET N4_FLGPASAMARCOPAUTA_MOD_AJ = 
	CASE   WHEN CODSUBSEGMENTO NOT IN ('X1N','LEN','P1N') THEN
	(CASE WHEN (CODSUBSEGMENTO = 'M1N'
	                   AND  NUEVA_SEGMENTACION_2 = 'ARMANDO' AND TIPOINGRESOACTUAL IN     ('NPH0','NPH1','NPH2','NPH3','NPH4')) THEN 
	(CASE WHEN SCORE_FINAL >= 390 AND coalesce(N4_APALANCAMIENTO,0) <= 25 THEN 1
	                      WHEN SCORE_FINAL >= 360 AND coalesce(N4_APALANCAMIENTO,0) <= 15 THEN 1
	                      WHEN SCORE_FINAL >= 330 AND coalesce(N4_APALANCAMIENTO,0) <= 12 THEN 1
	                      WHEN SCORE_FINAL >= 315 AND coalesce(N4_APALANCAMIENTO,0) <= 8  THEN 1
	                      WHEN SCORE_FINAL >= 300 AND coalesce(N4_APALANCAMIENTO,0) <= 4  THEN 1
	                      ELSE 0 END)
	       WHEN (CODSUBSEGMENTO = 'M1N' 
	                   AND  NUEVA_SEGMENTACION_2 = 'ARMANDO' AND TIPOINGRESOACTUAL NOT IN ('NPH0','NPH1','NPH2','NPH3','NPH4')) THEN 
	(CASE WHEN SCORE_FINAL >= 390 AND coalesce(N4_APALANCAMIENTO,0) <= 25 THEN 1
	                      WHEN SCORE_FINAL >= 360 AND coalesce(N4_APALANCAMIENTO,0) <= 15 THEN 1
	                      WHEN SCORE_FINAL >= 330 AND coalesce(N4_APALANCAMIENTO,0) <= 12 THEN 1
	                      WHEN SCORE_FINAL >= 315 AND coalesce(N4_APALANCAMIENTO,0) <= 6  THEN 1
	                      WHEN SCORE_FINAL >= 300 AND coalesce(N4_APALANCAMIENTO,0) <= 2  THEN 1
	                      ELSE 0 END)  
	       ELSE
	       CASE 
	                   WHEN SCORE_FINAL >= 390 AND coalesce(N4_APALANCAMIENTO,0) <= 25 THEN 1
	                   WHEN SCORE_FINAL >= 360 AND coalesce(N4_APALANCAMIENTO,0) <= 15 THEN 1
	                   WHEN SCORE_FINAL >= 330 AND coalesce(N4_APALANCAMIENTO,0) <= 12 THEN 1
	                   WHEN SCORE_FINAL >= 315 AND coalesce(N4_APALANCAMIENTO,0) <= 8  THEN 1
	                   WHEN SCORE_FINAL >= 300 AND coalesce(N4_APALANCAMIENTO,0) <= 4  THEN 1
	                   ELSE 0 
	       END                 
	END)
	WHEN CODSUBSEGMENTO IN ('LEN','P1N') THEN 
	       CASE WHEN SCORE_FINAL >= 390 AND coalesce(N4_APALANCAMIENTO,0) <= 25  THEN 1
	                   WHEN SCORE_FINAL >= 340 AND coalesce(N4_APALANCAMIENTO,0) <= 10  THEN 1
	                   WHEN SCORE_FINAL >= 300 AND coalesce(N4_APALANCAMIENTO,0) <= 5   THEN 1
	                   ELSE 0 END
	WHEN CODSUBSEGMENTO IN ('X1N') THEN 1
	END;
"""

raw_update_statement6 = """
UPDATE (SELECT * FROM TP_AGENDA_DIARIA_3 WHERE FLG_VULNERABLE = 0)X
		SET
		X.FLG_CLIENTESTOP=CASE WHEN X.PRIORIDAD_CLIENTE_TOP<=C.PERCAPITA/2 THEN 'S' ELSE 'N' END
		WHERE
		X.MAT_ADV=C.MATRICULA;
"""

from src.main.datatrace.base.statements.update_statement import UpdateStatement
from src.main.datatrace.base.statements.select_statement import BasicSelectStatement, SelectStatement
from sqlglot import parse_one, exp

update_statement = UpdateStatement(raw_update_statement5)
components = update_statement.get_components()
output_table = update_statement.get_output_table()
output_table_alias = update_statement.get_output_table_alias()
field_mapping = update_statement.get_field_mapping()

# print("Components:", components)
raw_upd_stm = update_statement.parse_statement()
print(raw_upd_stm)
print(raw_upd_stm.find_all(exp.Subquery))

lista = []
for s in raw_upd_stm.find_all(exp.Subquery):
    print("hola:",s)
    print(s.sql(dialect="oracle"))
    lista.append(SelectStatement(s.sql(dialect="oracle"),update_statement.statement_id,update_statement.output_table).get_field_mapping())
print(lista)
    

UPDATE PARALELA_FT_CEFCNV_TABLON SET N4_FLGPASAMARCOPAUTA_MOD_AJ = CASE WHEN NOT CODSUBSEGMENTO IN ('X1N', 'LEN', 'P1N') THEN (CASE WHEN (CODSUBSEGMENTO = 'M1N' AND NUEVA_SEGMENTACION_2 = 'ARMANDO' AND TIPOINGRESOACTUAL IN ('NPH0', 'NPH1', 'NPH2', 'NPH3', 'NPH4')) THEN (CASE WHEN SCORE_FINAL >= 390 AND COALESCE(N4_APALANCAMIENTO, 0) <= 25 THEN 1 WHEN SCORE_FINAL >= 360 AND COALESCE(N4_APALANCAMIENTO, 0) <= 15 THEN 1 WHEN SCORE_FINAL >= 330 AND COALESCE(N4_APALANCAMIENTO, 0) <= 12 THEN 1 WHEN SCORE_FINAL >= 315 AND COALESCE(N4_APALANCAMIENTO, 0) <= 8 THEN 1 WHEN SCORE_FINAL >= 300 AND COALESCE(N4_APALANCAMIENTO, 0) <= 4 THEN 1 ELSE 0 END) WHEN (CODSUBSEGMENTO = 'M1N' AND NUEVA_SEGMENTACION_2 = 'ARMANDO' AND NOT TIPOINGRESOACTUAL IN ('NPH0', 'NPH1', 'NPH2', 'NPH3', 'NPH4')) THEN (CASE WHEN SCORE_FINAL >= 390 AND COALESCE(N4_APALANCAMIENTO, 0) <= 25 THEN 1 WHEN SCORE_FINAL >= 360 AND COALESCE(N4_APALANCAMIENTO, 0) <= 15 THEN 1 WHEN SCORE_FINAL >= 330 AND COALESCE(N4_APALANCAMIENTO, 0) <= 

In [140]:
import sqlglot
from sqlglot import exp

def extract_ast_info(sql):
    parsed = sqlglot.parse_one(sql)
    return extract_node_info(parsed)

def extract_node_info(node):
    info = {}
    if isinstance(node, exp.Update):
        info['type'] = 'UPDATE'
        info['output_table'] = node.this.name if node.this else None
        info['set_clauses'] = [extract_node_info(exp) for exp in node.expressions]
        info['from_clause'] = extract_node_info(node.args.get('from')) if node.args.get('from') else None
        info['where_clause'] = extract_node_info(node.args.get('where')) if node.args.get('where') else None
    elif isinstance(node, exp.Select):
        info['type'] = 'SELECT'
        info['columns'] = [extract_node_info(exp) for exp in node.expressions]
        info['from_clause'] = extract_node_info(node.args.get('from')) if node.args.get('from') else None
        info['where_clause'] = extract_node_info(node.args.get('where')) if node.args.get('where') else None
    elif isinstance(node, exp.Table):
        info['type'] = 'TABLE'
        info['name'] = node.this.name if node.this else None
        info['db'] = node.db.name if node.db else None
    elif isinstance(node, exp.Column):
        info['type'] = 'COLUMN'
        info['name'] = node.this.name if node.this else None
    elif isinstance(node, exp.Literal):
        info['type'] = 'LITERAL'
        info['value'] = node.this
    elif isinstance(node, exp.EQ):
        info['type'] = 'EQ'
        info['left'] = extract_node_info(node.this)
        info['right'] = extract_node_info(node.expression)
    elif isinstance(node, exp.CurrentTimestamp):
        info['type'] = 'CURRENT_TIMESTAMP'
    else:
        info['type'] = type(node).__name__
        info['value'] = str(node)
    return info

# Ejemplo de uso
raw_update_statement = """
UPDATE (SELECT * FROM MD_LEADSAGENDA B WHERE B.PRODUCTO IN('CREDITO EFECTIVO','BALANCE TRANSFER','CUENTA SUELDO','PRESTAMO TARJETERO','TARJETA DE CREDITO')) A
  SET A.PRODUCTO = 'Este cliente es tu CASERO y tiene un A.PRODUCTO APROVÉCHALO!'
  WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD','CASERO VENTA','CUADERNITO VIRTUAL'));
"""

ast_info = extract_ast_info(raw_update_statement)
print(ast_info)

{'type': 'UPDATE', 'output_table': '', 'set_clauses': [{'type': 'EQ', 'left': {'type': 'COLUMN', 'name': 'PRODUCTO'}, 'right': {'type': 'LITERAL', 'value': 'Este cliente es tu CASERO y tiene un A.PRODUCTO APROVÉCHALO!'}}], 'from_clause': None, 'where_clause': {'type': 'Where', 'value': "WHERE A.CODCLAVECIC IN (SELECT DISTINCT CODCLAVECIC FROM MM_LEADSAGENDADIARIA WHERE TIPO_ASIGNACION IN ('CASERO SOLICITUD', 'CASERO VENTA', 'CUADERNITO VIRTUAL'))"}}


In [8]:
import src.main.datatrace.base.params as params
import src.main.datatrace.utils.utils as utils

raw_statement = """CREATE TABLE USRDB_CRM_SOLTEC.UM_SCORETARJETAAPP AS

SELECT
B.CODCLAVECIC,  
A.SC_TCSOLAF AS NUMSCOREAPPTC,
TRUNC(SYSDATE) AS FECACTUALIZACIONTABLA,
CODMES AS CODMES,
A.NUMSEGMENTO
FROM 
(
SELECT * FROM T43739.v_Hm_Score_Newapp_Univ 
WHERE CODMES=(SELECT MAX(CODMES) FROM T43739.v_Hm_Score_Newapp_Univ)
) A	
LEFT JOIN ODS_V.MD_CLIENTEG94 B
ON A.CODCLAVECIC=B.CODCLAVECIC AND B.FLGREGELIMINADO='N'
WHERE B.CODCLAVECIC IS NOT NULL;"""

components_dict = utils.extract_regex_as_dict(params.STATEMENT_STRUCT_DICT["CREATE_TABLE_AS"],
                                                      raw_statement)

print(components_dict)

{'raw_output_table': 'USRDB_CRM_SOLTEC.UM_SCORETARJETAAPP', 'tablespace': 'AS\n\nSELECT\nB.CODCLAVECIC,  \nA.SC_TCSOLAF', 'raw_select_statement': "NUMSCOREAPPTC,\nTRUNC(SYSDATE) AS FECACTUALIZACIONTABLA,\nCODMES AS CODMES,\nA.NUMSEGMENTO\nFROM \n(\nSELECT * FROM T43739.v_Hm_Score_Newapp_Univ \nWHERE CODMES=(SELECT MAX(CODMES) FROM T43739.v_Hm_Score_Newapp_Univ)\n) A\t\nLEFT JOIN ODS_V.MD_CLIENTEG94 B\nON A.CODCLAVECIC=B.CODCLAVECIC AND B.FLGREGELIMINADO='N'\nWHERE B.CODCLAVECIC IS NOT NULL", 'flag_success': 1}
